In [3]:
# 🚀 FLUJO COMPLETO DE ENTRENAMIENTO CNN PARA PREDICCIÓN DE CLIPS 🚀

# ✅ 1. IMPORTAR LIBRERÍAS NECESARIAS
import os
import numpy as np
import pandas as pd
import cv2 as cv
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# ✅ 2. CONFIGURAR PARÁMETROS GLOBALES
IMG_SIZE = (128, 128)  # Tamaño para las imágenes
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# ✅ 3. RUTAS DE DATOS
ruta_train_csv = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train.csv'
ruta_test_csv = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test.csv'
ruta_train_img = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train'
ruta_test_img = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test'

# ✅ 4. CARGAR LOS CSV
df_train = pd.read_csv(ruta_train_csv)
df_test = pd.read_csv(ruta_test_csv)

# ✅ 5. PREPROCESAR IMÁGENES
def preprocesar_imagen(img_path):
    img = cv.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen: {img_path}")
    
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    mask_azul = cv.inRange(hsv, (100, 50, 50), (140, 255, 255))
    mask_rojo1 = cv.inRange(hsv, (0, 50, 50), (10, 255, 255))
    mask_rojo2 = cv.inRange(hsv, (170, 50, 50), (180, 255, 255))
    mask_rojo = cv.bitwise_or(mask_rojo1, mask_rojo2)
    mask_combined = cv.bitwise_or(mask_azul, mask_rojo)
    img[mask_combined > 0] = (200, 200, 200)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    _, thresh = cv.threshold(gray, 180, 255, cv.THRESH_BINARY)
    return cv.resize(cv.bitwise_not(thresh), IMG_SIZE)

def cargar_datos(df, ruta_img):
    X = []
    for img_id in tqdm(df['id'], desc='Procesando imágenes'):
        img_path = os.path.join(ruta_img, f'clips-{img_id}.png')
        X.append(preprocesar_imagen(img_path))
    return np.array(X, dtype=np.float32) / 255.0

X_train = cargar_datos(df_train, ruta_train_img)
X_test = cargar_datos(df_test, ruta_test_img)

X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]
y_train = df_train['clip_count'].values.astype('float32')

# ✅ 6. DIVIDIR EN TRAIN Y VALIDATION
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, shuffle=True
)

# ✅ 7. DEFINIR MODELO CNN
modelo = Sequential([
    Input(shape=(128, 128, 1)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1)  # Salida para regresión
])

modelo.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='mean_squared_error',
    metrics=['mae']
)

# ✅ 8. CALLBACKS
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
callback_checkpoint = ModelCheckpoint('modelo_mejor.keras', monitor='val_loss', save_best_only=True, mode='min')

# ✅ 9. ENTRENAR MODELO
historial = modelo.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=BATCH_SIZE,
    callbacks=[callback_early_stopping, callback_reduce_lr, callback_checkpoint]
)

# ✅ 10. EVALUAR MODELO
loss, mae = modelo.evaluate(X_val, y_val)
print(f'✅ Pérdida en Validación: {loss:.4f}, MAE: {mae:.4f}')

# ✅ 11. VISUALIZAR RESULTADOS
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(historial.history['loss'], label='Pérdida Entrenamiento')
plt.plot(historial.history['val_loss'], label='Pérdida Validación')
plt.legend()
plt.title('Evolución de la Pérdida')

plt.subplot(1, 2, 2)
plt.plot(historial.history['mae'], label='MAE Entrenamiento')
plt.plot(historial.history['val_mae'], label='MAE Validación')
plt.legend()
plt.title('Evolución del MAE')
plt.show()

# ✅ 12. PREDICCIONES EN TEST Y GUARDAR RESULTADOS
y_pred = modelo.predict(X_test)
df_test['clip_count'] = y_pred.flatten().astype(int)
df_test.to_csv('resultados_test.csv', index=False)
print("✅ Resultados guardados en 'resultados_test.csv'")


Procesando imágenes: 100%|██████████| 5000/5000 [00:11<00:00, 423.92it/s]


Epoch 1/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 75s 195ms/step - loss: 375.2496 - mae: 14.9743 - val_loss: 1094.1676 - val_mae: 27.8300 - learning_rate: 1.0000e-04
Epoch 2/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 83s 222ms/step - loss: 132.8349 - mae: 8.6518 - val_loss: 73.0405 - val_mae: 6.8598 - learning_rate: 1.0000e-04
Epoch 3/50
118/375 ━━━━━━━━━━━━━━━━━━━━ 1:00 236ms/step - loss: 115.5236 - mae: 8.0271

KeyboardInterrupt: 